In [1]:
import os
import pandas as pd
path = "results/synthetic_data"
folders = ["results/synthetic_data/10%",
           "results/synthetic_data/25%",
           "results/synthetic_data/50%"
          ]
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    for file in files:
        if 'cfeval' in file:
            df = pd.read_csv(os.path.join(subdir,file))
            df['alphabet_coverage'] = subdir.split('/')[-1]
            dfs.append(df)
df1 = pd.concat(dfs)


path = "results/bpic2012_O_ACCEPTED-COMPLETE"
folders = ["results/bpic2012_O_ACCEPTED-COMPLETE/10%",
           "results/bpic2012_O_ACCEPTED-COMPLETE/25%",
           "results/bpic2012_O_ACCEPTED-COMPLETE/50%"
          ] 
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    for file in files:
        if 'cfeval' in file:
            df = pd.read_csv(os.path.join(subdir,file))
            df['alphabet_coverage'] = subdir.split('/')[-1]
            dfs.append(df)
df2 = pd.concat(dfs)



path = "results/BPIC17_O_ACCEPTED/"
folders = ["results/BPIC17_O_ACCEPTED/10%",
           "results/BPIC17_O_ACCEPTED/25%",
           "results/BPIC17_O_ACCEPTED/50%"
          ]
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    for file in files:
        if 'cfeval' in file:
            df = pd.read_csv(os.path.join(subdir,file))
            df['alphabet_coverage'] = subdir.split('/')[-1]
            dfs.append(df)
df3 = pd.concat(dfs)

df1.loc[df1['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'
df2.loc[df2['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'
df3.loc[df3['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'

df1.loc[df1['heuristic'] == 'mar', 'heuristic'] = 'mutate_retry_baseline'
df2.loc[df2['heuristic'] == 'mar', 'heuristic'] = 'mutate_retry_baseline'
df3.loc[df3['heuristic'] == 'mar', 'heuristic'] = 'mutate_retry_baseline'

ValueError: No objects to concatenate

In [ ]:
import pandas as pd
from scipy.stats import friedmanchisquare, wilcoxon
import statsmodels.stats.multitest as smm
import numpy as np

# Assuming df1, df2, df3 are your DataFrames for the three datasets
dfs = [df1, df2, df3]
results = []

# Preprocess the data and calculate mean values for grouping
for i, df in enumerate(dfs, 1):
    df_groupby = df.groupby(['heuristic', 'prefix_length', 'desired_nr_of_cfs', 'alphabet_coverage']).mean(numeric_only=True).reset_index()
    df_groupby['dataset'] = df['dataset'].iloc[0]
    results.append(df_groupby)

# Combine all datasets into a single DataFrame
combined_df = pd.concat(results)
combined_df = combined_df.rename(columns={'sat_score':'conformance', 'distance_l2j':'distance', 
                                          'implausibility_sum':'implausibility', 'avg_nbr_changes_per_cf':'sparsity', 
                                          'diversity_l2j':'diversity'})
combined_df['hit_rate'] = combined_df['generated_cfs'] / combined_df['desired_nr_of_cfs']

# List of metrics
metrics = ['distance', 'sparsity', 'implausibility', 'conformance', 'diversity', 'hit_rate', 'runtime']

friedman_results = {}
pairwise_results = {}

# Perform tests for each dataset and alphabet_coverage
for dataset in combined_df['dataset'].unique():
    dataset_df = combined_df[combined_df['dataset'] == dataset]
    friedman_results[dataset] = {}
    pairwise_results[dataset] = {}

    for alphabet_coverage in dataset_df['alphabet_coverage'].unique():
        ac_df = dataset_df[dataset_df['alphabet_coverage'] == alphabet_coverage]
        friedman_results[dataset][alphabet_coverage] = {}
        pairwise_results[dataset][alphabet_coverage] = {}

        for metric in metrics:
            # Ensure uniqueness by grouping and averaging
            df_pivot = ac_df.groupby(['prefix_length', 'desired_nr_of_cfs', 'heuristic'])[metric].mean().unstack()

            # Drop rows with any NaN values to ensure equal N for all groups
            df_pivot = df_pivot.dropna()

            if len(df_pivot.columns) < 2:
                continue  # Skip if there are less than 2 methods to compare

            # Perform Friedman test
            stat, p_value = friedmanchisquare(*[df_pivot[method] for method in df_pivot.columns])
            friedman_results[dataset][alphabet_coverage][metric] = p_value

            if p_value < 0.05:
                heuristics = df_pivot.columns
                p_values = []
                pairs = []

                for i in range(len(heuristics)):
                    for j in range(i + 1, len(heuristics)):
                        h1 = heuristics[i]
                        h2 = heuristics[j]
                        data_h1 = df_pivot[h1]
                        data_h2 = df_pivot[h2]

                        if np.array_equal(data_h1, data_h2):
                            p_value = 1.0
                        else:
                            try:
                                # Perform Wilcoxon signed-rank test
                                stat, p_value = wilcoxon(data_h1, data_h2, zero_method='wilcox')
                            except ValueError:
                                p_value = np.nan
                        p_values.append(p_value)
                        pairs.append((h1, h2))

                # Perform multiple comparisons correction (e.g., Bonferroni correction)
                corrected_p_vals = smm.multipletests(p_values, alpha=0.05, method='bonferroni')[1]
                pairwise_results[dataset][alphabet_coverage][metric] = dict(zip(pairs, corrected_p_vals))

# Initialize a dictionary to store ranks
ranks = {dataset: {alphabet_coverage: {metric: {} for metric in metrics} 
                   for alphabet_coverage in combined_df['alphabet_coverage'].unique()} 
         for dataset in combined_df['dataset'].unique()}

# Assign ranks based on significant differences
for dataset in combined_df['dataset'].unique():
    for alphabet_coverage in combined_df['alphabet_coverage'].unique():
        dataset_ac_df = combined_df[(combined_df['dataset'] == dataset) & 
                                    (combined_df['alphabet_coverage'] == alphabet_coverage)]

        for metric in metrics:
            df_pivot = dataset_ac_df.groupby(['prefix_length', 'desired_nr_of_cfs', 'heuristic'])[metric].mean().unstack()
            df_pivot = df_pivot.dropna()

            if len(df_pivot.columns) < 2:
                continue  # Skip if there are less than 2 methods to compare

            methods = df_pivot.columns

            if friedman_results[dataset][alphabet_coverage][metric] < 0.05:
                for method in methods:
                    ranks[dataset][alphabet_coverage][metric][method] = 1  # Initial rank

                for (h1, h2), corrected_p_val in pairwise_results[dataset][alphabet_coverage][metric].items():
                    if corrected_p_val < 0.05:
                        if df_pivot[h1].mean() < df_pivot[h2].mean():
                            ranks[dataset][alphabet_coverage][metric][h2] += 1
                        else:
                            ranks[dataset][alphabet_coverage][metric][h1] += 1
            else:
                for method in methods:
                    ranks[dataset][alphabet_coverage][metric][method] = 1  # Same rank if no significant difference

# Create a DataFrame to store the ranks
rank_table = pd.DataFrame(columns=['dataset', 'metric', 'alphabet_coverage', 'heuristic', 'rank'])

# Populate the DataFrame with ranks
for dataset in combined_df['dataset'].unique():
    for alphabet_coverage in combined_df['alphabet_coverage'].unique():
        dataset_ac_df = combined_df[(combined_df['dataset'] == dataset) & 
                                    (combined_df['alphabet_coverage'] == alphabet_coverage)]
        for metric in metrics:
            df_pivot = dataset_ac_df.groupby(['prefix_length', 'desired_nr_of_cfs', 'heuristic'])[metric].mean().unstack()
            df_pivot = df_pivot.dropna()
            for (prefix_length, desired_nr_of_cfs), row in df_pivot.iterrows():
                for heuristic, value in row.items():
                    rank_table = rank_table.append({
                        'dataset': dataset,
                        'metric': metric,
                        'alphabet_coverage': alphabet_coverage,
                        'heuristic': heuristic,
                        'rank': ranks[dataset][alphabet_coverage][metric][heuristic]
                    }, ignore_index=True)

# Pivot the rank_table to show ranks for each dataset, heuristic, metric, and alphabet coverage


In [ ]:
pivot_table = rank_table.pivot_table(index=['dataset', 'alphabet_coverage','metric'], columns='heuristic', values='rank')


In [ ]:
pivot_table.columns

In [ ]:
from IPython.core.display import HTML
pivot_table = pivot_table[['genetic_ltlf_baseline_operators','mutate_retry_baseline', 'heuristic_1',
       'heuristic_2']]
display(HTML(pivot_table.to_html()))

In [ ]:
# Assuming df1, df2, df3 are your DataFrames for the three datasets
dfs = [df1, df2, df3]
results = []
for i, df in enumerate(dfs, 1):
    df_groupby = df.groupby(['heuristic', 'prefix_length', 'desired_nr_of_cfs']).mean(numeric_only=True).reset_index()
    df_groupby['dataset'] = df['dataset'][1].unique()[0]
    results.append(df_groupby)


# Combine all datasets into a single DataFrame
combined_df = pd.concat(results)
combined_df = pd.concat(results)
combined_df = combined_df.rename(columns={'sat_score':'conformance','distance_l2j':'distance','implausibility_sum':'implausibility'
                                       ,'avg_nbr_changes_per_cf':'sparsity','diversity_l2j':'diversity'})
combined_df['hit_rate'] = combined_df['generated_cfs']/combined_df['desired_nr_of_cfs']
# List of metrics
metrics = ['distance', 'sparsity', 'implausibility', 'conformance', 'diversity', 'hit_rate', 'runtime']

friedman_results = {}
pairwise_results = {}

# Perform tests for each dataset
for dataset in combined_df['dataset'].unique():
    dataset_df = combined_df[combined_df['dataset'] == dataset]
    friedman_results[dataset] = {}
    pairwise_results[dataset] = {}

    for metric in metrics:
        df_pivot = dataset_df.pivot(index=['prefix_length', 'desired_nr_of_cfs'], columns='heuristic', values=metric)

        # Drop rows with any NaN values to ensure equal N for all groups
        df_pivot = df_pivot.dropna()

        if len(df_pivot.columns) < 2:
            continue  # Skip if there are less than 2 methods to compare

        # Perform Friedman test
        stat, p_value = friedmanchisquare(*[df_pivot[method] for method in df_pivot.columns])
        friedman_results[dataset][metric] = p_value

        if p_value < 0.05:
            heuristics = df_pivot.columns
            p_values = []
            pairs = []

            for i in range(len(heuristics)):
                for j in range(i + 1, len(heuristics)):
                    h1 = heuristics[i]
                    h2 = heuristics[j]
                    data_h1 = df_pivot[h1]
                    data_h2 = df_pivot[h2]

                    if np.array_equal(data_h1, data_h2):
                        p_value = 1.0
                    else:
                        # Perform Wilcoxon signed-rank test
                        try:
                            stat, p_value = wilcoxon(data_h1, data_h2, zero_method='wilcox')
                        except ValueError:
                            p_value = np.nan
                    p_values.append(p_value)
                    pairs.append((h1, h2))

            # Perform multiple comparisons correction (e.g., Bonferroni correction)
            corrected_p_vals = smm.multipletests(p_values, alpha=0.05, method='bonferroni')[1]
            pairwise_results[dataset][metric] = dict(zip(pairs, corrected_p_vals))

# Initialize a dictionary to store ranks
ranks = {dataset: {metric: {} for metric in metrics} for dataset in combined_df['dataset'].unique()}

# Assign ranks based on significant differences
for dataset in combined_df['dataset'].unique():
    dataset_df = combined_df[combined_df['dataset'] == dataset]

    for metric in metrics:
        df_pivot = dataset_df.pivot(index=['prefix_length', 'desired_nr_of_cfs'], columns='heuristic', values=metric)

        # Drop rows with any NaN values to ensure equal N for all groups
        df_pivot = df_pivot.dropna()

        if len(df_pivot.columns) < 2:
            continue  # Skip if there are less than 2 methods to compare

        methods = df_pivot.columns

        if friedman_results[dataset][metric] < 0.05:
            for method in methods:
                ranks[dataset][metric][method] = 1  # Initial rank

            for (h1, h2), corrected_p_val in pairwise_results[dataset][metric].items():
                if corrected_p_val < 0.05:
                    if df_pivot[h1].mean() < df_pivot[h2].mean():
                        ranks[dataset][metric][h2] += 1
                    else:
                        ranks[dataset][metric][h1] += 1
        else:
            for method in methods:
                ranks[dataset][metric][method] = 1  # Same rank if no significant difference

# Create a DataFrame to store the ranks
rank_table = pd.DataFrame(columns=['dataset', 'metric', 'heuristic', 'rank'])

# Populate the DataFrame with ranks
for dataset in ranks.keys():
    for metric in ranks[dataset].keys():
        for method, rank in ranks[dataset][metric].items():
            rank_table = rank_table.append({
                'dataset': dataset,
                'metric': metric,
                'heuristic': method,
                'rank': rank
            }, ignore_index=True)

# Pivot the rank_table to show ranks for each dataset, heuristic, and metric
pivot_table = rank_table.pivot_table(index=['dataset', 'heuristic'], columns='metric', values='rank')

# Display the pivot table


In [ ]:
pivot_table

# Dataset wise boxplot results

In [ ]:
import matplotlib.patches as mpatches
import os
import pandas as pd
import matplotlib.pyplot as plt
path = "results/synthetic_data"
folders = ["results/synthetic_data/10%",
           "results/synthetic_data/25%",
           "results/synthetic_data/50%"]
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    for file in files:
        if 'cfeval' in file:
            df = pd.read_csv(os.path.join(subdir,file))
            df['alphabet_coverage'] = subdir.split('/')[-1]
            dfs.append(df)
final_df = pd.concat(dfs)


prefixes = [7,9,11,13]
df = final_df
# Update 'heuristic' column where 'method' is 'genetic'
df.loc[df['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'

# Update 'heuristic' column where 'method' is 'random'

df = df[df['prefix_length'].isin(prefixes)]
df = df.rename(columns={'sat_score':'trace fitness','distance_l2j':'distance','implausibility_sum':'implausibility'
                                       ,'avg_nbr_changes_per_cf':'sparsity','diversity_l2j':'diversity'})
all_data = df
all_data['hit_rate'] = all_data['generated_cfs']/all_data['desired_nr_of_cfs']

rows = 0 
coverages = ["10%","25%","50%"]
fig_1, axes = plt.subplots(nrows=len(coverages), ncols=7, figsize=(35, 16),sharey='col')

for coverage in coverages:
    df = all_data[all_data['alphabet_coverage'].str.contains(coverage)]
    #df['optimization'].replace({'multiobjective_adapted':'AOMO','multiobjective_baseline':'BOMO',
    #                           'single_adapted':'AOSO','single_baseline':'BOSO'},inplace=True)
    df = df[
        ['heuristic','prefix_length','desired_nr_of_cfs','distance', 'sparsity','implausibility','trace fitness','diversity','hit_rate','runtime']]
    df_groupby = df.groupby(['heuristic', 'prefix_length','desired_nr_of_cfs']).mean(numeric_only=True)
    # crate the figure and axes
    #df_groupby.iloc[-32:-16,-1] *= 0.3
    for j, i in enumerate(df_groupby.columns):
        #df_groupby[[i]].unstack(level=0).plot(ax=axes[rows][j], legend=None, title=i,linewidth=5.0,
        #                                      style=['--', '-', '--','-'],
                                              #cmap="Accent",
        #                                      fontsize=18,color=['red','orange','green','blue']
        #                                  )
        medianprops = dict(linewidth=3, color='black')
        bplot = df_groupby[[i]].boxplot(by='heuristic',ax=axes[rows][j], #legend=None, title=i,#,linewidth=5.0,
                                      #style=['--', '-', '--','-'],
                                      #cmap="Accent",
                                      patch_artist=True,fontsize=18,medianprops=medianprops,
                                         return_type='both'#,color=['red','orange','green','blue']
                                  )
        colors=['red','orange','green','gray','magenta']
        for row_key, (ax,row) in bplot.items():
            ax.set_xlabel('')
            for i,box in enumerate(row['boxes']):
                box.set_facecolor(colors[i])


    for ax in axes.flatten():
        ax.yaxis.set_tick_params(labelleft=True,labelsize=22)
        ax.set(xlabel=None)
        ax.set(xticklabels=[])
        ax.set_title(label=ax.title._text,fontsize=22,fontweight='bold')
    axes[rows,3].set_title("Alphabet coverage "+coverage+"\n"+'satisfaction rate',fontweight="bold",size=22)
    axes[rows,3].set_ylim(0.3,1.1)
    axes[rows,5].set_ylim(0.3,1.1)

    rows += 1
red_patch = mpatches.Patch(color='red', label='genetic',)
orange_patch = mpatches.Patch(color='orange', label='genetic_ltlf_baseline_operators')
green_patch = mpatches.Patch(color='green', label='h1.Outback')
#blue_patch = mpatches.Patch(color='blue', label='h2.Highlander')
gray_patch = mpatches.Patch(color='gray', label='h2.Pathfinder')
magenta_patch = mpatches.Patch(color='magenta', label='h3.Freelander')
fig_1.legend(handles=[red_patch,orange_patch,green_patch,gray_patch,magenta_patch],loc='upper center',
        bbox_to_anchor=(0.5, 1.03),
        ncol=6,prop={"size":27},fontsize=40)
fig_1.suptitle('Claim Management',x=0.07,y=0.73,fontsize=40,fontweight='bold',rotation=90)
#fig_1.savefig('/Users/andrei/Desktop/PhD/boxplots_overall_ecai_paper_synthetic_log.png', bbox_inches='tight')


In [ ]:
import matplotlib.patches as mpatches
import os
import pandas as pd
import matplotlib.pyplot as plt
path = "results/bpic2012_O_ACCEPTED-COMPLETE"
folders = ["results/bpic2012_O_ACCEPTED-COMPLETE/10%",
           "results/bpic2012_O_ACCEPTED-COMPLETE/25%",
           "results/bpic2012_O_ACCEPTED-COMPLETE/50%"
          ]
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    if subdir in folders:
        for file in files:
                if 'cfeval' in file:
                        df = pd.read_csv(os.path.join(subdir,file))
                        df['alphabet_coverage'] = subdir.split('/')[-1]
                        dfs.append(df)
final_df = pd.concat(dfs)


prefixes = [20,25,30,35]
df = final_df
# Update 'heuristic' column where 'method' is 'genetic'
df.loc[df['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'

df = df[df['prefix_length'].isin(prefixes)]
df = df.rename(columns={'sat_score':'trace fitness','distance_l2j':'distance','implausibility_sum':'implausibility'
                                       ,'avg_nbr_changes_per_cf':'sparsity','diversity_l2j':'diversity'})
all_data = df
all_data['hit_rate'] = all_data['generated_cfs']/all_data['desired_nr_of_cfs']

rows = 0 
coverages = ["10%","25%",
             "50%"
            ]
fig_1, axes = plt.subplots(nrows=len(coverages), ncols=7, figsize=(35, 16),sharey='col')

for coverage in coverages:
    df = all_data[all_data['alphabet_coverage'].str.contains(coverage)]
    #df['optimization'].replace({'multiobjective_adapted':'AOMO','multiobjective_baseline':'BOMO',
    #                           'single_adapted':'AOSO','single_baseline':'BOSO'},inplace=True)
    df = df[
        ['heuristic','prefix_length','desired_nr_of_cfs','distance', 'sparsity','implausibility','trace fitness','diversity','hit_rate','runtime']]
    df_groupby = df.groupby(['heuristic', 'prefix_length','desired_nr_of_cfs']).mean(numeric_only=True)
    # crate the figure and axes
    #df_groupby.iloc[-32:-16,-1] *= 0.3
    for j, i in enumerate(df_groupby.columns):
        #df_groupby[[i]].unstack(level=0).plot(ax=axes[rows][j], legend=None, title=i,linewidth=5.0,
        #                                      style=['--', '-', '--','-'],
                                              #cmap="Accent",
        #                                      fontsize=18,color=['red','orange','green','blue']
        #                                  )
        medianprops = dict(linewidth=3, color='black')
        bplot = df_groupby[[i]].boxplot(by='heuristic',ax=axes[rows][j], #legend=None, title=i,#,linewidth=5.0,
                                      #style=['--', '-', '--','-'],
                                      #cmap="Accent",
                                      patch_artist=True,fontsize=18,medianprops=medianprops,
                                         return_type='both'#,color=['red','orange','green','blue']
                                  )
        colors=['red','orange','green','gray','magenta']
        for row_key, (ax,row) in bplot.items():
            ax.set_xlabel('')
            for i,box in enumerate(row['boxes']):
                box.set_facecolor(colors[i])


    for ax in axes.flatten():
        ax.yaxis.set_tick_params(labelleft=True,labelsize=22)
        ax.set(xlabel=None)
        ax.set(xticklabels=[])
        ax.set_title(label=ax.title._text,fontsize=22,fontweight='bold')
    axes[rows,3].set_title("Alphabet coverage "+coverage+"\n"+'satisfaction rate',fontweight="bold",size=22)
    axes[rows,3].set_ylim(0.3,1.1)
    axes[rows,5].set_ylim(0.3,1.1)

    rows += 1
red_patch = mpatches.Patch(color='red', label='genetic',)
orange_patch = mpatches.Patch(color='orange', label='genetic_ltlf_baseline_operators')
green_patch = mpatches.Patch(color='green', label='h1.Outback')
#blue_patch = mpatches.Patch(color='blue', label='h2.Highlander')
gray_patch = mpatches.Patch(color='gray', label='h2.Pathfinder')
magenta_patch = mpatches.Patch(color='magenta', label='h3.Freelander')
fig_1.legend(handles=[red_patch,orange_patch,green_patch,gray_patch,magenta_patch],loc='upper center',
        bbox_to_anchor=(0.5, 1.03),
        ncol=6,prop={"size":27},fontsize=40)
fig_1.suptitle('BPIC2012',x=0.07,y=0.73,fontsize=40,fontweight='bold',rotation=90)
#fig_1.savefig('/Users/andrei/Desktop/PhD/boxplots_overall_ecai_paper_synthetic_log.png', bbox_inches='tight')


In [ ]:
import matplotlib.patches as mpatches
import os
import pandas as pd
import matplotlib.pyplot as plt
path = "results/BPIC17_O_ACCEPTED"
folders = ["results/BPIC17_O_ACCEPTED/10%",
           "results/BPIC17_O_ACCEPTED/25%",
           "results/BPIC17_O_ACCEPTED/50%"
          ]
dfs = []
for subdir, dirs, files in os.walk(path+'/'):
    if subdir in folders:
        for file in files:
                if 'cfeval' in file:
                        df = pd.read_csv(os.path.join(subdir,file))
                        df['alphabet_coverage'] = subdir.split('/')[-1]
                        dfs.append(df)
final_df = pd.concat(dfs)


prefixes = [20,25,30,35]
df = final_df
# Update 'heuristic' column where 'method' is 'genetic'
df.loc[df['method'] == 'genetic_ltlf_baseline_operators', 'heuristic'] = 'genetic_ltlf_baseline_operators'
# Update 'heuristic' column where 'method' is 'random'

df = df[df['prefix_length'].isin(prefixes)]
df = df.rename(columns={'sat_score':'trace fitness','distance_l2j':'distance','implausibility_sum':'implausibility'
                                       ,'avg_nbr_changes_per_cf':'sparsity','diversity_l2j':'diversity'})
all_data = df
all_data['hit_rate'] = all_data['generated_cfs']/all_data['desired_nr_of_cfs']

rows = 0 
coverages = ["10%","25%",
             "50%"
            ]
fig_1, axes = plt.subplots(nrows=len(coverages), ncols=7, figsize=(35, 16),sharey='col')

for coverage in coverages:
    df = all_data[all_data['alphabet_coverage'].str.contains(coverage)]
    #df['optimization'].replace({'multiobjective_adapted':'AOMO','multiobjective_baseline':'BOMO',
    #                           'single_adapted':'AOSO','single_baseline':'BOSO'},inplace=True)
    df = df[
        ['heuristic','prefix_length','desired_nr_of_cfs','distance', 'sparsity','implausibility','trace fitness','diversity','hit_rate','runtime']]
    df_groupby = df.groupby(['heuristic', 'prefix_length','desired_nr_of_cfs']).mean(numeric_only=True)
    # crate the figure and axes
    #df_groupby.iloc[-32:-16,-1] *= 0.3
    for j, i in enumerate(df_groupby.columns):
        #df_groupby[[i]].unstack(level=0).plot(ax=axes[rows][j], legend=None, title=i,linewidth=5.0,
        #                                      style=['--', '-', '--','-'],
                                              #cmap="Accent",
        #                                      fontsize=18,color=['red','orange','green','blue']
        #                                  )
        medianprops = dict(linewidth=3, color='black')
        bplot = df_groupby[[i]].boxplot(by='heuristic',ax=axes[rows][j], #legend=None, title=i,#,linewidth=5.0,
                                      #style=['--', '-', '--','-'],
                                      #cmap="Accent",
                                      patch_artist=True,fontsize=18,medianprops=medianprops,
                                         return_type='both'#,color=['red','orange','green','blue']
                                  )
        colors=['red','orange','green','gray','magenta']
        for row_key, (ax,row) in bplot.items():
            ax.set_xlabel('')
            for i,box in enumerate(row['boxes']):
                box.set_facecolor(colors[i])


    for ax in axes.flatten():
        ax.yaxis.set_tick_params(labelleft=True,labelsize=22)
        ax.set(xlabel=None)
        ax.set(xticklabels=[])
        ax.set_title(label=ax.title._text,fontsize=22,fontweight='bold')
    axes[rows,3].set_title("Alphabet coverage "+coverage+"\n"+'satisfaction rate',fontweight="bold",size=22)
    axes[rows,3].set_ylim(0.3,1.1)
    axes[rows,5].set_ylim(0.3,1.1)

    rows += 1
red_patch = mpatches.Patch(color='red', label='genetic',)
orange_patch = mpatches.Patch(color='orange', label='genetic_ltlf_baseline_operators')
green_patch = mpatches.Patch(color='green', label='h1.Outback')
#blue_patch = mpatches.Patch(color='blue', label='h2.Highlander')
gray_patch = mpatches.Patch(color='gray', label='h2.Pathfinder')
magenta_patch = mpatches.Patch(color='magenta', label='h3.Freelander')
fig_1.legend(handles=[red_patch,orange_patch,green_patch,gray_patch,magenta_patch],loc='upper center',
        bbox_to_anchor=(0.5, 1.03),
        ncol=6,prop={"size":27},fontsize=40)
fig_1.suptitle('BPIC17_O_ACCEPTED',x=0.07,y=0.73,fontsize=40,fontweight='bold',rotation=90)
#fig_1.savefig('/Users/andrei/Desktop/PhD/boxplots_overall_ecai_paper_synthetic_log.png', bbox_inches='tight')
